In [1]:
import pandas as pd
import numpy as np

# Генератор имен

Генератор набора данных для имен.

In [ ]:
m_names = pd.read_excel("data/names/train_data.xlsx", "masc_names")['М_Имя'].to_list()
m_surnames = pd.read_excel("data/names/train_data.xlsx", "masc_surnames")['М_Фамилии'].to_list()
m_patnames = pd.read_excel("data/names/train_data.xlsx", "masc_patnames")['М_Отчества'].to_list()
m_names_arrays = {"names": m_names, 
                  "surnames": m_surnames, 
                  "patronymics": m_patnames}

f_names = pd.read_excel("data/names/train_data.xlsx", "fem_names")['Ж_Имя'].to_list()
f_surnames = pd.read_excel("data/names/train_data.xlsx", "fem_surnames")['Ж_Фамилии'].to_list()
f_patnames = pd.read_excel("data/names/train_data.xlsx", "fem_patnames")["Ж_Отчетства"].to_list()
f_names_arrays = {"names": f_names, 
                  "surnames": f_surnames, 
                  "patronymics": f_patnames}

In [ ]:
ner_tags = ['O', 'PER-NAME', 'PER_SURN', 'PER-PATR']

In [ ]:
def compose_name() -> tuple:

    name = dict()
    namesList = []

    # генерируем параметры для имени
    sex = np.random.choice(["m", "f"], 1)[0]

    # print(sex)

    n_surnames = np.random.choice(6, size=1, p = [0.01, 0.5, 0.2, 0.2, 0.05, 0.04])[0]
    add_name = np.random.choice(2, size = 1, p = [0.05, 0.95])[0]
    add_patronymic = np.random.choice(2, size = 1, p = [0.05, 0.95])[0]

    if n_surnames == 0 and add_name == 0 and add_patronymic == 0:
        n_surnames = 1
        add_name = 1
        add_patronymic = 1

    namesparts_order = np.array([0]*add_name + [1]*n_surnames + [2]*add_patronymic) + 1
    np.random.shuffle(namesparts_order)

    # print(namesparts_order)

    if sex == "m":
        names_arrays = m_names_arrays
    else:
        names_arrays = f_names_arrays

    name[0] = np.random.choice(names_arrays["names"], size = add_name)

    name[1] = np.random.choice(names_arrays["surnames"], n_surnames, False)

    name[2] = np.random.choice(names_arrays["patronymics"], size = add_patronymic)

    for idx in namesparts_order:

        if idx == 1:
            namesList.append(name[0][0])
        
        if idx == 2:
            namesList.append(name[1][n_surnames - 1])
            n_surnames -= 1

        if idx == 3:
            namesList.append(name[2][0])

    return((namesList, namesparts_order))

In [ ]:
dataset = pd.DataFrame([compose_name() for i in range(15000)], columns=["tokens", "ner_tags"])

In [ ]:
dataset.to_json("data/names/names_train.json")
dataset.head()

# Генератор адресов

In [3]:
from simpledbf import Dbf5

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [15]:
dbf = Dbf5('data/addresses/base/KLADR.DBF', 'cp866')
adr_data = dbf.to_dataframe()

adr_data.head()

,NAME,SOCR,CODE,INDEX,GNINMB,UNO,OCATD,STATUS
0,Адыгея,Респ,0100000000000,NaN,0100,NaN,79000000000,0
1,Майкоп,г,0100000100000,NaN,0100,NaN,79401000000,0
2,Веселый,х,0100000100200,NaN,0100,NaN,79401000006,0
3,Гавердовский,х,0100000100300,NaN,0100,NaN,79401000002,0
4,Подгорный,п,0100000100400,NaN,0100,NaN,79401000004,0


In [22]:
adr_data["REGIONCODE"] = adr_data["CODE"].str[:2]

In [38]:
adr_data[adr_data['OCATD'] == "79205000000"]

,NAME,SOCR,CODE,INDEX,GNINMB,UNO,OCATD,STATUS,REGIONCODE
418,Гиагинский,р-н,0100800000000,NaN,0100,0101,79205000000,0,01


In [35]:
dbf = Dbf5('data/addresses/base/SOCRBASE.DBF', 'cp866')
dbf_data = dbf.to_dataframe()

dbf_data

,LEVEL,SCNAME,SOCRNAME,KOD_T_ST
0,1,Аобл,Автономная область,102
1,1,АО,Автономный округ,101
2,1,г,Город,103
3,1,г.ф.з.,Город федерального значения,112
4,1,край,Край,104
...,...,...,...,...
223,5,ф/х,Фермерское хозяйство,589
224,5,х,Хутор,558
225,5,ш,Шоссе,531
226,5,ю.,Юрты,931


In [46]:
dbf_data[dbf_data['LEVEL'] == "1"]

,LEVEL,SCNAME,SOCRNAME,KOD_T_ST
0,1,Аобл,Автономная область,102
1,1,АО,Автономный округ,101
2,1,г,Город,103
3,1,г.ф.з.,Город федерального значения,112
4,1,край,Край,104
5,1,обл,Область,105
6,1,округ,Округ,107
7,1,Респ,Республика,106
8,1,Чувашия,Чувашия,108
